In [1]:
import keras
import numpy as np
import pandas as pd

In [2]:
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
train,test = train_test_split(df,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [5]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()

#validation dataset
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel() 

In [6]:
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)



In [7]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [8]:
train_x.shape[1]

11

In [15]:
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],momentum=params['momentum']
    ),
    loss = "mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ##training the ANN model with lr and momentum parameters with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return { "loss" : eval_rmse , "status" : STATUS_OK , "model" : model }

In [10]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [11]:
space = {
    "lr": hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [16]:
mlflow.set_experiment("/wine_quality")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn = objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )   

    best_run = sorted(trials.results,key=lambda x: x['loss'])[0]
    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 43.1098 - root_mean_squared_error: 6.5652
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - loss: 42.0828 - root_mean_squared_error: 6.5652
 1/46 ━━━━━━━━━━━━━━━━━━━━ 43s 959ms/step - loss: 41.3601 - root_mean_squared_error: 6.5336
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 42.6942 - root_mean_squared_error: 6.5336
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43.1982 - root_mean_squared_error: 6.5720
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 43.2416 - root_mean_squared_error: 6.5754
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 43.1960 - root_mean_squared_error: 6.5722
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 43.1794 - root_mean_squared_error: 6.5693
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 43.1342 - root_mean_squared_error: 6.5675
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 43.1578 - root_mean_squared_error: 6.5675
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 43.102

In [18]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/24c1ee8cc8374311b15c63cb9d570825/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\Meet\OneDrive\Desktop\engg\MLFlow\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[4.153991 ],
       [7.387936 ],
       [6.4068494],
       ...,
       [6.902651 ],
       [6.8939614],
       [4.753352 ]], dtype=float32)

In [19]:
model2 = mlflow.pyfunc.load_model(model_uri)
model2.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[4.153991 ],
       [7.387936 ],
       [6.4068494],
       ...,
       [6.902651 ],
       [6.8939614],
       [4.753352 ]], dtype=float32)